In [1]:
from datetime import datetime
register_input_path = "/data/students/bigdata_internet/lab3/register.csv"
stations_input_path = "/data/students/bigdata_internet/lab3/stations.csv"
outputPath = "lab3DF"

In [2]:
# Create a DataFrame from register.csv
register_df = spark.read.load(register_input_path,
                     format="csv", header=True, inferSchema=True, sep="\t")

In [3]:
# Create a DataFrame from stations.csv
stations_df = spark.read.load(stations_input_path,
                     format="csv", header=True, inferSchema=True, sep="\t")

In [ ]:
# total number of lines in the register data frame
register_df.count()

In [10]:
# I need to remove the lines of the register_df that have errors
# removing the erroneous lines where both used and free slots are zero 
filtered_register_df = register_df.filter("not(used_slots=0 and free_slots=0)")

In [11]:
filtered_register_df.count()

25104121

In [12]:
def create_timeslot(timestamp):
    # '2008-05-15 12:20:00' is the timestamp format and is a timestamp object of class datetime
    weekday = (timestamp.strftime('%A'))
    hour = (timestamp.strftime("%H"))
    timeslot = str(weekday) +' '+ str(hour)
    return timeslot

In [42]:
spark.udf.register("toWeekday", lambda timestamp: (timestamp.strftime('%A')) )

In [43]:
spark.udf.register("toHour", lambda timestamp: timestamp.strftime('%H'))

In [44]:
spark.udf.register("toTimeslot", lambda timestamp: create_timeslot(timestamp))

In [16]:
register_w_weekday_hour_df = filtered_register_df.selectExpr("station","toWeekday(timestamp) as weekday","toHour(timestamp) as hour", "used_slots","free_slots" )

In [17]:
timestamp_df = filtered_register_df.selectExpr("station","toTimeslot(timestamp) as timeslot", "used_slots","free_slots" )

In [18]:
def checkcriticality(free_slots):
    critical = 0
    if free_slots == 0: # do not have to cast to int since df infer schema already makes it integer
        critical = 1
    else :
        critical = 0
    return int(critical)

In [19]:
def countreading(station):
    return int(1)

In [45]:
spark.udf.register("CountReading", lambda station: countreading(station) ,"integer")

In [46]:
spark.udf.register("isCritical", lambda free_slots: checkcriticality(free_slots) ,"integer")

In [22]:
criticality_df = timestamp_df.selectExpr("station","timeslot","isCritical(free_slots) as critical","CountReading(station) as reading")

In [23]:
statio_timeslot_criticaltotal_df = criticality_df.groupBy("station","timeslot").sum("critical","reading").withColumnRenamed("sum(critical)","total_critical")\
                                                                                                                            .withColumnRenamed("sum(reading)","total_readings")

In [24]:
statio_timeslot_criticaltotal_df.show(2)

+-------+-----------+--------------+--------------+
|station|   timeslot|total_critical|total_readings|
+-------+-----------+--------------+--------------+
|    180|  Friday 18|             0|           597|
|    180|Thursday 08|             3|           531|
+-------+-----------+--------------+--------------+
only showing top 2 rows



In [25]:
def calculateCriticality(total_critical,total_readings):
    criticality = float(total_critical/total_readings)
    return criticality

In [47]:
spark.udf.register("CalculateCriticality", lambda total_critical,total_readings: calculateCriticality(total_critical,total_readings) ,"float")

In [27]:
station_timeslot_criticality_df = statio_timeslot_criticaltotal_df.selectExpr("station","timeslot","CalculateCriticality(total_critical,total_readings) as criticality")

In [28]:
#station_timeslot_criticality_df.show(5)

In [29]:
# Defining a threshold to filter the values with 
THRESHOLD = 0.6

In [30]:
filtered_criticality_df = station_timeslot_criticality_df.filter("criticality>{}".format(THRESHOLD))

In [31]:
filtered_criticality_df.show(2)

+-------+-----------+-----------+
|station|   timeslot|criticality|
+-------+-----------+-----------+
|      9|  Friday 10| 0.61290324|
|     10|Saturday 00|   0.622108|
+-------+-----------+-----------+
only showing top 2 rows



In [32]:
filtered_criticality_df.count()

5

In [33]:
sorted_criticality_df = filtered_criticality_df.sort("criticality",ascending = False)

In [34]:
sorted_criticality_df.show(5)

+-------+-----------+-----------+
|station|   timeslot|criticality|
+-------+-----------+-----------+
|     58|  Monday 00|   0.632312|
|      9|  Friday 22| 0.62583894|
|     58|  Monday 01|  0.6239554|
|     10|Saturday 00|   0.622108|
|      9|  Friday 10| 0.61290324|
+-------+-----------+-----------+



In [35]:
#  ta.join(tb, ta.name == tb.name, 'inner'). 
sorted_criticality_w_coordinates_df = sorted_criticality_df.join(stations_df,sorted_criticality_df.station == stations_df.id, "inner")

In [41]:
#sorted_criticality_w_coordinates_df.show(2)

In [37]:
final_df = sorted_criticality_w_coordinates_df.select("station",  "timeslot", "criticality",  "longitude" ,"latitude")

In [38]:
final_df.show(5)

+-------+-----------+-----------+---------+---------+
|station|   timeslot|criticality|longitude| latitude|
+-------+-----------+-----------+---------+---------+
|     58|  Monday 00|   0.632312| 2.170736|41.377536|
|      9|  Friday 22| 0.62583894| 2.185294|41.385006|
|     58|  Monday 01|  0.6239554| 2.170736|41.377536|
|     10|Saturday 00|   0.622108| 2.185206|41.384875|
|      9|  Friday 10| 0.61290324| 2.185294|41.385006|
+-------+-----------+-----------+---------+---------+



In [40]:
final_df.write.csv(outputPath, header=True,sep = "\t")